In [348]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/zindi-ai/Test_v2.csv
/kaggle/input/zindi-ai/Train_v2.csv
/kaggle/input/ai-zindi-competion/Test_v2.csv
/kaggle/input/ai-zindi-competion/Train_v2.csv


In [349]:
#import catboost

In [350]:
%matplotlib notebook

In [351]:
import numpy as np
import pandas as pd
from pprint import pprint
from collections import Counter

In [352]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [353]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier

from mlxtend.classifier import EnsembleVoteClassifier

In [354]:
import xgboost as xgb
from catboost import CatBoostClassifier

In [360]:
Test = pd.read_csv("/kaggle/input/ai-zindi-competion/Test_v2.csv")

In [361]:
Test.head()

,country,year,uniqueid,location_type,cellphone_access,household_size,age_of_respondent,gender_of_respondent,relationship_with_head,marital_status,education_level,job_type
0,Kenya,2018,uniqueid_6056,Urban,Yes,3,30,Male,Head of Household,Married/Living together,Secondary education,Formally employed Government
1,Kenya,2018,uniqueid_6060,Urban,Yes,7,51,Male,Head of Household,Married/Living together,Vocational/Specialised training,Formally employed Private
2,Kenya,2018,uniqueid_6065,Rural,No,3,77,Female,Parent,Married/Living together,No formal education,Remittance Dependent
3,Kenya,2018,uniqueid_6072,Rural,No,6,39,Female,Head of Household,Married/Living together,Primary education,Remittance Dependent
4,Kenya,2018,uniqueid_6073,Urban,No,3,16,Male,Child,Single/Never Married,Secondary education,Remittance Dependent


In [362]:
Train = pd.read_csv("/kaggle/input/ai-zindi-competion/Train_v2.csv")

In [363]:
Train.head()

,country,year,uniqueid,bank_account,location_type,cellphone_access,household_size,age_of_respondent,gender_of_respondent,relationship_with_head,marital_status,education_level,job_type
0,Kenya,2018,uniqueid_1,Yes,Rural,Yes,3,24,Female,Spouse,Married/Living together,Secondary education,Self employed
1,Kenya,2018,uniqueid_2,No,Rural,No,5,70,Female,Head of Household,Widowed,No formal education,Government Dependent
2,Kenya,2018,uniqueid_3,Yes,Urban,Yes,5,26,Male,Other relative,Single/Never Married,Vocational/Specialised training,Self employed
3,Kenya,2018,uniqueid_4,No,Rural,Yes,5,34,Female,Head of Household,Married/Living together,Primary education,Formally employed Private
4,Kenya,2018,uniqueid_5,No,Urban,No,8,26,Male,Child,Single/Never Married,Primary education,Informally employed


In [364]:
Test.head(1)

,country,year,uniqueid,location_type,cellphone_access,household_size,age_of_respondent,gender_of_respondent,relationship_with_head,marital_status,education_level,job_type
0,Kenya,2018,uniqueid_6056,Urban,Yes,3,30,Male,Head of Household,Married/Living together,Secondary education,Formally employed Government


In [365]:
Train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23524 entries, 0 to 23523
Data columns (total 13 columns):
country                   23524 non-null object
year                      23524 non-null int64
uniqueid                  23524 non-null object
bank_account              23524 non-null object
location_type             23524 non-null object
cellphone_access          23524 non-null object
household_size            23524 non-null int64
age_of_respondent         23524 non-null int64
gender_of_respondent      23524 non-null object
relationship_with_head    23524 non-null object
marital_status            23524 non-null object
education_level           23524 non-null object
job_type                  23524 non-null object
dtypes: int64(3), object(10)
memory usage: 2.3+ MB


In [366]:
for column in Train.columns:
    print(Train[column].unique())

['Kenya' 'Rwanda' 'Tanzania' 'Uganda']
[2018 2016 2017]
['uniqueid_1' 'uniqueid_2' 'uniqueid_3' ... 'uniqueid_8757'
 'uniqueid_8758' 'uniqueid_8759']
['Yes' 'No']
['Rural' 'Urban']
['Yes' 'No']
[ 3  5  8  7  1  6  4 10  2 11  9 12 16 15 13 14 21 18 17 20]
[ 24  70  26  34  32  42  54  76  40  69  64  31  38  47  27  48  25  21
  18  22  58  55  62  29  35  45  67  19  80  66  50  33  28  51  16  17
  30  37  59  65  46  56  52  23  43  49  44  72  53  63  39  81  78  36
  20  60  95  71  57  85  68  41  61  75  86  73  93  74  88  90  77  84
  82  89  79  83  94  87  92  91  98  97  96  99 100]
['Female' 'Male']
['Spouse' 'Head of Household' 'Other relative' 'Child' 'Parent'
 'Other non-relatives']
['Married/Living together' 'Widowed' 'Single/Never Married'
 'Divorced/Seperated' 'Dont know']
['Secondary education' 'No formal education'
 'Vocational/Specialised training' 'Primary education'
 'Tertiary education' 'Other/Dont know/RTA']
['Self employed' 'Government Dependent' 'Formally em

In [367]:
for column in Test.columns:
    print(Test[column].unique())

['Kenya' 'Rwanda' 'Tanzania' 'Uganda']
[2018 2016 2017]
['uniqueid_6056' 'uniqueid_6060' 'uniqueid_6065' ... 'uniqueid_3000'
 'uniqueid_3001' 'uniqueid_3002']
['Urban' 'Rural']
['Yes' 'No']
[ 3  7  6  4  1  5  2  9 10 11  8 12 13 16 15 14 18 20]
[ 30  51  77  39  16  25  32  24  22  42  18  20  19  26  35  68  70  34
  95  27  21  45  52  44  75  60  33  36  53  64  66  23  78  38  46  76
  82  41  28  17  50  48  55  37  57  91  87  56  40  59  73  47  74  43
  79  49  58  31  54  63  29  62  90  84  72  65  80  61  67  69  86  85
  71  83  81  93  88  94  89  99  92 100  97]
['Male' 'Female']
['Head of Household' 'Parent' 'Child' 'Spouse' 'Other relative'
 'Other non-relatives']
['Married/Living together' 'Single/Never Married' 'Divorced/Seperated'
 'Widowed' 'Dont know']
['Secondary education' 'Vocational/Specialised training'
 'No formal education' 'Primary education' 'Other/Dont know/RTA'
 'Tertiary education']
['Formally employed Government' 'Formally employed Private'
 'Remittan

In [368]:
Columns_Transform = ["country","location_type","cellphone_access","age_of_respondent",
                     "gender_of_respondent","relationship_with_head","marital_status","education_level",
                     "job_type"]

In [369]:
encode_label = {}
for column in Columns_Transform:
    encode_label[column] = LabelEncoder().fit(Train[column])
    

In [370]:
Train_Copy = Train.copy()

In [371]:
for column in Columns_Transform:
    Train_Copy[column] = encode_label[column].transform(Train_Copy[column])

In [372]:
Train_Copy.head()

,country,year,uniqueid,bank_account,location_type,cellphone_access,household_size,age_of_respondent,gender_of_respondent,relationship_with_head,marital_status,education_level,job_type
0,0,2018,uniqueid_1,Yes,0,1,3,8,0,5,2,3,9
1,0,2018,uniqueid_2,No,0,0,5,54,0,1,4,0,4
2,0,2018,uniqueid_3,Yes,1,1,5,10,1,3,3,5,9
3,0,2018,uniqueid_4,No,0,1,5,18,0,1,2,2,3
4,0,2018,uniqueid_5,No,1,0,8,10,1,0,3,2,5


In [373]:
Train_Copy.columns

Index(['country', 'year', 'uniqueid', 'bank_account', 'location_type',
       'cellphone_access', 'household_size', 'age_of_respondent',
       'gender_of_respondent', 'relationship_with_head', 'marital_status',
       'education_level', 'job_type'],
      dtype='object')

In [374]:
feature_columns = ['country', 'year','location_type',
       'cellphone_access', 'household_size', 'age_of_respondent',
       'gender_of_respondent', 'relationship_with_head', 'marital_status',
       'education_level', 'job_type']

In [375]:
target_encoder = LabelEncoder().fit(Train_Copy['bank_account'])
Train_Copy['bank_account'] = target_encoder.transform(Train_Copy['bank_account'])

In [376]:
Train_Copy['bank_account'].head(5)

0    1
1    0
2    1
3    0
4    0
Name: bank_account, dtype: int64

In [377]:
X_train, X_test, y_train, y_test = train_test_split( Train_Copy[feature_columns], Train_Copy['bank_account'], test_size=0.2, random_state=42)

In [378]:
lr_model = LogisticRegression()
gb_model = GradientBoostingClassifier()
svc_model = LinearSVC()
rf_model = RandomForestClassifier()
ada_model = AdaBoostClassifier()

In [379]:
lr_model = lr_model.fit(X_train, y_train)
lr_pred = lr_model.predict(X_test)
accuracy = accuracy_score(y_test, lr_pred)

print(accuracy)
error_rate = 1 - accuracy
print(error_rate)

0.8790648246546228
0.12093517534537723


/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [380]:
gb_model = gb_model.fit(X_train, y_train)
gb_pred = gb_model.predict(X_test)
accuracy = accuracy_score(y_test, gb_pred)

print(accuracy)
error_rate = 1 - accuracy
print( error_rate)

0.8907545164718385
0.10924548352816155


In [381]:
rf_model = rf_model.fit(X_train, y_train)
rf_pred = rf_model.predict(X_test)
accuracy = accuracy_score(y_test, rf_pred)

print(accuracy)
error_rate = 1 - accuracy
print(error_rate)

0.8620616365568544
0.13793836344314558


/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [382]:
ada_model = AdaBoostClassifier(learning_rate=0.6)
ada_model = ada_model.fit(X_train, y_train)
ada_pred = ada_model.predict(X_test)
accuracy = accuracy_score(y_test, ada_pred)

print(accuracy)
error_rate = 1 - accuracy
print(error_rate)

0.8886291179596174
0.11137088204038259


In [383]:
Test_Copy = Test.copy()

for column in Columns_Transform:
    Test_Copy[column] = encode_label[column].transform(Test_Copy[column])

In [384]:
xgb_model = xgb.XGBClassifier().fit(X_train, y_train)
xgb_pred = xgb_model.predict(X_test)
accuracy = accuracy_score(y_test, xgb_pred)
print(accuracy)
error_rate = 1 - accuracy
print(error_rate)

0.8905419766206164
0.1094580233793836


In [385]:
catmodel = CatBoostClassifier(iterations=5000)
catmodel = catmodel.fit(X_train, y_train)
cat_pred = catmodel.predict(X_test)

Learning rate set to 0.009163
0:	learn: 0.6834842	total: 56.7ms	remaining: 4m 43s
1:	learn: 0.6753174	total: 91.5ms	remaining: 3m 48s
2:	learn: 0.6673583	total: 134ms	remaining: 3m 42s
3:	learn: 0.6600522	total: 172ms	remaining: 3m 34s
4:	learn: 0.6519846	total: 209ms	remaining: 3m 29s
5:	learn: 0.6442378	total: 248ms	remaining: 3m 26s
6:	learn: 0.6365890	total: 283ms	remaining: 3m 21s
7:	learn: 0.6280034	total: 320ms	remaining: 3m 19s
8:	learn: 0.6211822	total: 360ms	remaining: 3m 19s
9:	learn: 0.6138253	total: 395ms	remaining: 3m 17s
10:	learn: 0.6069417	total: 433ms	remaining: 3m 16s
11:	learn: 0.6002326	total: 471ms	remaining: 3m 15s
12:	learn: 0.5925916	total: 508ms	remaining: 3m 15s
13:	learn: 0.5865098	total: 546ms	remaining: 3m 14s
14:	learn: 0.5803305	total: 582ms	remaining: 3m 13s
15:	learn: 0.5746253	total: 620ms	remaining: 3m 13s
16:	learn: 0.5679436	total: 658ms	remaining: 3m 13s
17:	learn: 0.5614468	total: 694ms	remaining: 3m 12s
18:	learn: 0.5559745	total: 731ms	remainin

157:	learn: 0.3117942	total: 6.53s	remaining: 3m 20s
158:	learn: 0.3113821	total: 6.57s	remaining: 3m 20s
159:	learn: 0.3110340	total: 6.61s	remaining: 3m 19s
160:	learn: 0.3105560	total: 6.65s	remaining: 3m 19s
161:	learn: 0.3102012	total: 6.7s	remaining: 3m 19s
162:	learn: 0.3098274	total: 6.74s	remaining: 3m 19s
163:	learn: 0.3095625	total: 6.77s	remaining: 3m 19s
164:	learn: 0.3091663	total: 6.81s	remaining: 3m 19s
165:	learn: 0.3088798	total: 6.86s	remaining: 3m 19s
166:	learn: 0.3085509	total: 6.89s	remaining: 3m 19s
167:	learn: 0.3082541	total: 6.94s	remaining: 3m 19s
168:	learn: 0.3080288	total: 6.99s	remaining: 3m 19s
169:	learn: 0.3077421	total: 7.04s	remaining: 3m 19s
170:	learn: 0.3074826	total: 7.08s	remaining: 3m 19s
171:	learn: 0.3070847	total: 7.12s	remaining: 3m 19s
172:	learn: 0.3067324	total: 7.16s	remaining: 3m 19s
173:	learn: 0.3064022	total: 7.22s	remaining: 3m 20s
174:	learn: 0.3061142	total: 7.26s	remaining: 3m 20s
175:	learn: 0.3057902	total: 7.3s	remaining: 3m

317:	learn: 0.2866496	total: 13.3s	remaining: 3m 16s
318:	learn: 0.2865821	total: 13.4s	remaining: 3m 16s
319:	learn: 0.2865312	total: 13.4s	remaining: 3m 16s
320:	learn: 0.2863974	total: 13.5s	remaining: 3m 16s
321:	learn: 0.2863099	total: 13.5s	remaining: 3m 16s
322:	learn: 0.2862590	total: 13.6s	remaining: 3m 16s
323:	learn: 0.2861663	total: 13.6s	remaining: 3m 16s
324:	learn: 0.2860774	total: 13.7s	remaining: 3m 16s
325:	learn: 0.2860238	total: 13.7s	remaining: 3m 16s
326:	learn: 0.2859798	total: 13.7s	remaining: 3m 16s
327:	learn: 0.2859553	total: 13.8s	remaining: 3m 16s
328:	learn: 0.2858727	total: 13.8s	remaining: 3m 16s
329:	learn: 0.2858299	total: 13.9s	remaining: 3m 16s
330:	learn: 0.2857844	total: 13.9s	remaining: 3m 16s
331:	learn: 0.2856876	total: 13.9s	remaining: 3m 16s
332:	learn: 0.2856553	total: 14s	remaining: 3m 15s
333:	learn: 0.2856350	total: 14s	remaining: 3m 15s
334:	learn: 0.2855412	total: 14.1s	remaining: 3m 15s
335:	learn: 0.2855085	total: 14.1s	remaining: 3m 1

476:	learn: 0.2798214	total: 19.9s	remaining: 3m 8s
477:	learn: 0.2797759	total: 19.9s	remaining: 3m 8s
478:	learn: 0.2797324	total: 20s	remaining: 3m 8s
479:	learn: 0.2796880	total: 20s	remaining: 3m 8s
480:	learn: 0.2796370	total: 20.1s	remaining: 3m 8s
481:	learn: 0.2795974	total: 20.1s	remaining: 3m 8s
482:	learn: 0.2795697	total: 20.1s	remaining: 3m 8s
483:	learn: 0.2795435	total: 20.2s	remaining: 3m 8s
484:	learn: 0.2795061	total: 20.2s	remaining: 3m 8s
485:	learn: 0.2795008	total: 20.3s	remaining: 3m 8s
486:	learn: 0.2794471	total: 20.3s	remaining: 3m 8s
487:	learn: 0.2794062	total: 20.3s	remaining: 3m 8s
488:	learn: 0.2793853	total: 20.4s	remaining: 3m 8s
489:	learn: 0.2793688	total: 20.4s	remaining: 3m 8s
490:	learn: 0.2793093	total: 20.5s	remaining: 3m 8s
491:	learn: 0.2792456	total: 20.5s	remaining: 3m 8s
492:	learn: 0.2792171	total: 20.6s	remaining: 3m 7s
493:	learn: 0.2791959	total: 20.6s	remaining: 3m 8s
494:	learn: 0.2791897	total: 20.7s	remaining: 3m 8s
495:	learn: 0.27

639:	learn: 0.2762679	total: 26.6s	remaining: 3m
640:	learn: 0.2762538	total: 26.6s	remaining: 3m
641:	learn: 0.2762449	total: 26.6s	remaining: 3m
642:	learn: 0.2762407	total: 26.7s	remaining: 3m
643:	learn: 0.2762134	total: 26.7s	remaining: 3m
644:	learn: 0.2762062	total: 26.8s	remaining: 3m
645:	learn: 0.2761889	total: 26.8s	remaining: 3m
646:	learn: 0.2761838	total: 26.8s	remaining: 3m
647:	learn: 0.2761709	total: 26.9s	remaining: 3m
648:	learn: 0.2761701	total: 26.9s	remaining: 3m
649:	learn: 0.2761616	total: 26.9s	remaining: 3m
650:	learn: 0.2761381	total: 27s	remaining: 3m
651:	learn: 0.2761220	total: 27s	remaining: 3m
652:	learn: 0.2761218	total: 27.1s	remaining: 3m
653:	learn: 0.2761123	total: 27.1s	remaining: 3m
654:	learn: 0.2760860	total: 27.1s	remaining: 3m
655:	learn: 0.2760736	total: 27.2s	remaining: 2m 59s
656:	learn: 0.2760702	total: 27.2s	remaining: 2m 59s
657:	learn: 0.2760689	total: 27.2s	remaining: 2m 59s
658:	learn: 0.2760580	total: 27.3s	remaining: 2m 59s
659:	lea

...

4217:	learn: 0.2622361	total: 2m 43s	remaining: 30.2s
4218:	learn: 0.2622360	total: 2m 43s	remaining: 30.2s
4219:	learn: 0.2622357	total: 2m 43s	remaining: 30.2s
4220:	learn: 0.2622357	total: 2m 43s	remaining: 30.1s
4221:	learn: 0.2622356	total: 2m 43s	remaining: 30.1s
4222:	learn: 0.2622356	total: 2m 43s	remaining: 30s
4223:	learn: 0.2622343	total: 2m 43s	remaining: 30s
4224:	learn: 0.2622340	total: 2m 43s	remaining: 30s
4225:	learn: 0.2622340	total: 2m 43s	remaining: 29.9s
4226:	learn: 0.2622337	total: 2m 43s	remaining: 29.9s
4227:	learn: 0.2622234	total: 2m 43s	remaining: 29.9s
4228:	learn: 0.2622233	total: 2m 43s	remaining: 29.8s
4229:	learn: 0.2622148	total: 2m 43s	remaining: 29.8s
4230:	learn: 0.2622145	total: 2m 43s	remaining: 29.7s
4231:	learn: 0.2622145	total: 2m 43s	remaining: 29.7s
4232:	learn: 0.2622136	total: 2m 43s	remaining: 29.7s
4233:	learn: 0.2622135	total: 2m 43s	remaining: 29.6s
4234:	learn: 0.2622126	total: 2m 43s	remaining: 29.6s
4235:	learn: 0.2622033	total: 2m 4

4375:	learn: 0.2619540	total: 2m 49s	remaining: 24.1s
4376:	learn: 0.2619539	total: 2m 49s	remaining: 24.1s
4377:	learn: 0.2619444	total: 2m 49s	remaining: 24.1s
4378:	learn: 0.2619443	total: 2m 49s	remaining: 24s
4379:	learn: 0.2619429	total: 2m 49s	remaining: 24s
4380:	learn: 0.2619429	total: 2m 49s	remaining: 23.9s
4381:	learn: 0.2619428	total: 2m 49s	remaining: 23.9s
4382:	learn: 0.2619328	total: 2m 49s	remaining: 23.9s
4383:	learn: 0.2619328	total: 2m 49s	remaining: 23.8s
4384:	learn: 0.2619328	total: 2m 49s	remaining: 23.8s
4385:	learn: 0.2619312	total: 2m 49s	remaining: 23.7s
4386:	learn: 0.2619312	total: 2m 49s	remaining: 23.7s
4387:	learn: 0.2619312	total: 2m 49s	remaining: 23.7s
4388:	learn: 0.2619311	total: 2m 49s	remaining: 23.6s
4389:	learn: 0.2619278	total: 2m 49s	remaining: 23.6s
4390:	learn: 0.2619273	total: 2m 49s	remaining: 23.5s
4391:	learn: 0.2619270	total: 2m 49s	remaining: 23.5s
4392:	learn: 0.2619256	total: 2m 49s	remaining: 23.5s
4393:	learn: 0.2619256	total: 2m

4531:	learn: 0.2617287	total: 2m 55s	remaining: 18.1s
4532:	learn: 0.2617286	total: 2m 55s	remaining: 18s
4533:	learn: 0.2617286	total: 2m 55s	remaining: 18s
4534:	learn: 0.2617286	total: 2m 55s	remaining: 18s
4535:	learn: 0.2617286	total: 2m 55s	remaining: 17.9s
4536:	learn: 0.2617224	total: 2m 55s	remaining: 17.9s
4537:	learn: 0.2617196	total: 2m 55s	remaining: 17.9s
4538:	learn: 0.2617196	total: 2m 55s	remaining: 17.8s
4539:	learn: 0.2617195	total: 2m 55s	remaining: 17.8s
4540:	learn: 0.2617164	total: 2m 55s	remaining: 17.7s
4541:	learn: 0.2617164	total: 2m 55s	remaining: 17.7s
4542:	learn: 0.2617164	total: 2m 55s	remaining: 17.7s
4543:	learn: 0.2617163	total: 2m 55s	remaining: 17.6s
4544:	learn: 0.2617163	total: 2m 55s	remaining: 17.6s
4545:	learn: 0.2617141	total: 2m 55s	remaining: 17.5s
4546:	learn: 0.2617141	total: 2m 55s	remaining: 17.5s
4547:	learn: 0.2617141	total: 2m 55s	remaining: 17.5s
4548:	learn: 0.2617141	total: 2m 55s	remaining: 17.4s
4549:	learn: 0.2617141	total: 2m 5

4689:	learn: 0.2615242	total: 3m	remaining: 11.9s
4690:	learn: 0.2615242	total: 3m	remaining: 11.9s
4691:	learn: 0.2615241	total: 3m	remaining: 11.9s
4692:	learn: 0.2615241	total: 3m	remaining: 11.8s
4693:	learn: 0.2615241	total: 3m	remaining: 11.8s
4694:	learn: 0.2615241	total: 3m	remaining: 11.8s
4695:	learn: 0.2615241	total: 3m	remaining: 11.7s
4696:	learn: 0.2615241	total: 3m 1s	remaining: 11.7s
4697:	learn: 0.2615238	total: 3m 1s	remaining: 11.6s
4698:	learn: 0.2615238	total: 3m 1s	remaining: 11.6s
4699:	learn: 0.2615218	total: 3m 1s	remaining: 11.6s
4700:	learn: 0.2615218	total: 3m 1s	remaining: 11.5s
4701:	learn: 0.2615192	total: 3m 1s	remaining: 11.5s
4702:	learn: 0.2615049	total: 3m 1s	remaining: 11.4s
4703:	learn: 0.2615049	total: 3m 1s	remaining: 11.4s
4704:	learn: 0.2615048	total: 3m 1s	remaining: 11.4s
4705:	learn: 0.2615048	total: 3m 1s	remaining: 11.3s
4706:	learn: 0.2614963	total: 3m 1s	remaining: 11.3s
4707:	learn: 0.2614963	total: 3m 1s	remaining: 11.3s
4708:	learn: 0

4849:	learn: 0.2612942	total: 3m 6s	remaining: 5.77s
4850:	learn: 0.2612895	total: 3m 6s	remaining: 5.74s
4851:	learn: 0.2612895	total: 3m 6s	remaining: 5.7s
4852:	learn: 0.2612894	total: 3m 6s	remaining: 5.66s
4853:	learn: 0.2612874	total: 3m 6s	remaining: 5.62s
4854:	learn: 0.2612874	total: 3m 6s	remaining: 5.58s
4855:	learn: 0.2612864	total: 3m 6s	remaining: 5.54s
4856:	learn: 0.2612864	total: 3m 6s	remaining: 5.5s
4857:	learn: 0.2612864	total: 3m 6s	remaining: 5.46s
4858:	learn: 0.2612863	total: 3m 7s	remaining: 5.43s
4859:	learn: 0.2612862	total: 3m 7s	remaining: 5.39s
4860:	learn: 0.2612827	total: 3m 7s	remaining: 5.35s
4861:	learn: 0.2612826	total: 3m 7s	remaining: 5.31s
4862:	learn: 0.2612761	total: 3m 7s	remaining: 5.27s
4863:	learn: 0.2612728	total: 3m 7s	remaining: 5.23s
4864:	learn: 0.2612728	total: 3m 7s	remaining: 5.2s
4865:	learn: 0.2612726	total: 3m 7s	remaining: 5.16s
4866:	learn: 0.2612726	total: 3m 7s	remaining: 5.12s
4867:	learn: 0.2612707	total: 3m 7s	remaining: 5.

In [386]:
accuracy = accuracy_score(y_test, cat_pred)
print("CatBoost5000", accuracy)
error_rate =1 - accuracy
print(error_rate)

CatBoost5000 0.8913921360255048
0.10860786397449518


In [387]:
Test_Copy.head()

,country,year,uniqueid,location_type,cellphone_access,household_size,age_of_respondent,gender_of_respondent,relationship_with_head,marital_status,education_level,job_type
0,0,2018,uniqueid_6056,1,1,3,14,1,1,2,3,2
1,0,2018,uniqueid_6060,1,1,7,35,1,1,2,5,3
2,0,2018,uniqueid_6065,0,0,3,61,0,4,2,0,8
3,0,2018,uniqueid_6072,0,0,6,23,0,1,2,2,8
4,0,2018,uniqueid_6073,1,0,3,0,1,0,3,3,8


In [388]:
catmodel1000 = CatBoostClassifier(depth=10)
catmodel1000 = catmodel1000.fit(X_train, y_train)
cat_pred = catmodel1000.predict(X_test)


Learning rate set to 0.038445
0:	learn: 0.6537561	total: 42.9ms	remaining: 42.8s
1:	learn: 0.6238533	total: 77.3ms	remaining: 38.6s
2:	learn: 0.5936197	total: 122ms	remaining: 40.4s
3:	learn: 0.5646520	total: 167ms	remaining: 41.5s
4:	learn: 0.5411612	total: 214ms	remaining: 42.6s
5:	learn: 0.5172989	total: 279ms	remaining: 46.2s
6:	learn: 0.4971562	total: 399ms	remaining: 56.6s
7:	learn: 0.4801943	total: 438ms	remaining: 54.3s
8:	learn: 0.4650779	total: 482ms	remaining: 53.1s
9:	learn: 0.4521721	total: 598ms	remaining: 59.2s
10:	learn: 0.4390264	total: 660ms	remaining: 59.3s
11:	learn: 0.4271226	total: 703ms	remaining: 57.9s
12:	learn: 0.4133285	total: 825ms	remaining: 1m 2s
13:	learn: 0.4045266	total: 870ms	remaining: 1m 1s
14:	learn: 0.3938899	total: 996ms	remaining: 1m 5s
15:	learn: 0.3844588	total: 1.06s	remaining: 1m 5s
16:	learn: 0.3745952	total: 1.11s	remaining: 1m 4s
17:	learn: 0.3669598	total: 1.16s	remaining: 1m 3s
18:	learn: 0.3613624	total: 1.21s	remaining: 1m 2s
19:	learn

161:	learn: 0.2691998	total: 10.4s	remaining: 53.8s
162:	learn: 0.2691830	total: 10.4s	remaining: 53.6s
163:	learn: 0.2690630	total: 10.5s	remaining: 53.8s
164:	learn: 0.2690407	total: 10.6s	remaining: 53.6s
165:	learn: 0.2688982	total: 10.7s	remaining: 53.8s
166:	learn: 0.2688797	total: 10.7s	remaining: 53.6s
167:	learn: 0.2688512	total: 10.8s	remaining: 53.4s
168:	learn: 0.2688183	total: 10.8s	remaining: 53.3s
169:	learn: 0.2688047	total: 10.9s	remaining: 53.1s
170:	learn: 0.2687027	total: 10.9s	remaining: 52.9s
171:	learn: 0.2686017	total: 11s	remaining: 52.8s
172:	learn: 0.2685137	total: 11s	remaining: 52.7s
173:	learn: 0.2685135	total: 11.1s	remaining: 52.5s
174:	learn: 0.2684935	total: 11.1s	remaining: 52.3s
175:	learn: 0.2683396	total: 11.2s	remaining: 52.5s
176:	learn: 0.2682904	total: 11.3s	remaining: 52.5s
177:	learn: 0.2682087	total: 11.3s	remaining: 52.3s
178:	learn: 0.2682054	total: 11.4s	remaining: 52.1s
179:	learn: 0.2681297	total: 11.4s	remaining: 52.1s
180:	learn: 0.26

322:	learn: 0.2639636	total: 18.6s	remaining: 39.1s
323:	learn: 0.2639633	total: 18.7s	remaining: 38.9s
324:	learn: 0.2639631	total: 18.7s	remaining: 38.8s
325:	learn: 0.2639483	total: 18.7s	remaining: 38.7s
326:	learn: 0.2639473	total: 18.8s	remaining: 38.6s
327:	learn: 0.2639260	total: 18.9s	remaining: 38.7s
328:	learn: 0.2639196	total: 18.9s	remaining: 38.6s
329:	learn: 0.2638763	total: 19s	remaining: 38.6s
330:	learn: 0.2638762	total: 19s	remaining: 38.4s
331:	learn: 0.2638761	total: 19s	remaining: 38.3s
332:	learn: 0.2638609	total: 19.1s	remaining: 38.3s
333:	learn: 0.2638590	total: 19.1s	remaining: 38.2s
334:	learn: 0.2637434	total: 19.3s	remaining: 38.2s
335:	learn: 0.2637431	total: 19.3s	remaining: 38.1s
336:	learn: 0.2637429	total: 19.3s	remaining: 38s
337:	learn: 0.2637386	total: 19.4s	remaining: 37.9s
338:	learn: 0.2637325	total: 19.4s	remaining: 37.8s
339:	learn: 0.2637322	total: 19.4s	remaining: 37.8s
340:	learn: 0.2637249	total: 19.5s	remaining: 37.7s
341:	learn: 0.263694

481:	learn: 0.2604143	total: 29.2s	remaining: 31.3s
482:	learn: 0.2604136	total: 29.2s	remaining: 31.3s
483:	learn: 0.2603806	total: 29.3s	remaining: 31.2s
484:	learn: 0.2603590	total: 29.4s	remaining: 31.2s
485:	learn: 0.2603246	total: 29.5s	remaining: 31.2s
486:	learn: 0.2603231	total: 29.6s	remaining: 31.2s
487:	learn: 0.2603181	total: 29.6s	remaining: 31.1s
488:	learn: 0.2602617	total: 29.8s	remaining: 31.1s
489:	learn: 0.2602340	total: 29.9s	remaining: 31.1s
490:	learn: 0.2601900	total: 30s	remaining: 31.1s
491:	learn: 0.2601873	total: 30s	remaining: 31s
492:	learn: 0.2601161	total: 30.2s	remaining: 31s
493:	learn: 0.2600254	total: 30.3s	remaining: 31s
494:	learn: 0.2600254	total: 30.3s	remaining: 30.9s
495:	learn: 0.2599975	total: 30.4s	remaining: 30.9s
496:	learn: 0.2599972	total: 30.5s	remaining: 30.8s
497:	learn: 0.2599942	total: 30.6s	remaining: 30.8s
498:	learn: 0.2598699	total: 30.7s	remaining: 30.8s
499:	learn: 0.2597500	total: 30.8s	remaining: 30.8s
500:	learn: 0.2597474	

642:	learn: 0.2570039	total: 41.7s	remaining: 23.1s
643:	learn: 0.2570037	total: 41.7s	remaining: 23s
644:	learn: 0.2570012	total: 41.7s	remaining: 23s
645:	learn: 0.2570002	total: 41.8s	remaining: 22.9s
646:	learn: 0.2569466	total: 41.9s	remaining: 22.9s
647:	learn: 0.2569452	total: 41.9s	remaining: 22.8s
648:	learn: 0.2569313	total: 42s	remaining: 22.7s
649:	learn: 0.2569286	total: 42.1s	remaining: 22.7s
650:	learn: 0.2569210	total: 42.2s	remaining: 22.6s
651:	learn: 0.2569210	total: 42.2s	remaining: 22.5s
652:	learn: 0.2569174	total: 42.3s	remaining: 22.5s
653:	learn: 0.2569125	total: 42.4s	remaining: 22.4s
654:	learn: 0.2569121	total: 42.4s	remaining: 22.3s
655:	learn: 0.2569121	total: 42.5s	remaining: 22.3s
656:	learn: 0.2569118	total: 42.5s	remaining: 22.2s
657:	learn: 0.2569118	total: 42.5s	remaining: 22.1s
658:	learn: 0.2569118	total: 42.6s	remaining: 22s
659:	learn: 0.2569113	total: 42.7s	remaining: 22s
660:	learn: 0.2569088	total: 42.7s	remaining: 21.9s
661:	learn: 0.2569084	

804:	learn: 0.2552332	total: 53.5s	remaining: 13s
805:	learn: 0.2552272	total: 53.7s	remaining: 12.9s
806:	learn: 0.2552272	total: 53.7s	remaining: 12.8s
807:	learn: 0.2552271	total: 53.7s	remaining: 12.8s
808:	learn: 0.2551541	total: 53.8s	remaining: 12.7s
809:	learn: 0.2551486	total: 54s	remaining: 12.7s
810:	learn: 0.2549494	total: 54.1s	remaining: 12.6s
811:	learn: 0.2549485	total: 54.1s	remaining: 12.5s
812:	learn: 0.2549069	total: 54.2s	remaining: 12.5s
813:	learn: 0.2548990	total: 54.4s	remaining: 12.4s
814:	learn: 0.2548967	total: 54.5s	remaining: 12.4s
815:	learn: 0.2548892	total: 54.5s	remaining: 12.3s
816:	learn: 0.2548849	total: 54.6s	remaining: 12.2s
817:	learn: 0.2548842	total: 54.6s	remaining: 12.2s
818:	learn: 0.2548532	total: 54.7s	remaining: 12.1s
819:	learn: 0.2548514	total: 54.8s	remaining: 12s
820:	learn: 0.2548514	total: 54.8s	remaining: 11.9s
821:	learn: 0.2548475	total: 54.9s	remaining: 11.9s
822:	learn: 0.2548473	total: 54.9s	remaining: 11.8s
823:	learn: 0.2548

967:	learn: 0.2531731	total: 1m 5s	remaining: 2.17s
968:	learn: 0.2531714	total: 1m 5s	remaining: 2.1s
969:	learn: 0.2531710	total: 1m 5s	remaining: 2.04s
970:	learn: 0.2531695	total: 1m 5s	remaining: 1.97s
971:	learn: 0.2531694	total: 1m 5s	remaining: 1.9s
972:	learn: 0.2531694	total: 1m 5s	remaining: 1.83s
973:	learn: 0.2531609	total: 1m 6s	remaining: 1.76s
974:	learn: 0.2531486	total: 1m 6s	remaining: 1.7s
975:	learn: 0.2531358	total: 1m 6s	remaining: 1.63s
976:	learn: 0.2531277	total: 1m 6s	remaining: 1.56s
977:	learn: 0.2531115	total: 1m 6s	remaining: 1.5s
978:	learn: 0.2531082	total: 1m 6s	remaining: 1.43s
979:	learn: 0.2531068	total: 1m 6s	remaining: 1.36s
980:	learn: 0.2531064	total: 1m 6s	remaining: 1.29s
981:	learn: 0.2531063	total: 1m 6s	remaining: 1.23s
982:	learn: 0.2531063	total: 1m 6s	remaining: 1.16s
983:	learn: 0.2531063	total: 1m 6s	remaining: 1.09s
984:	learn: 0.2531062	total: 1m 6s	remaining: 1.02s
985:	learn: 0.2530768	total: 1m 7s	remaining: 952ms
986:	learn: 0.25

In [389]:
accuracy = accuracy_score(y_test, cat_pred)
print("CatBoost (1000iterations):", accuracy)
error_rate =1 - accuracy
print(error_rate)

CatBoost (1000iterations): 0.8913921360255048
0.10860786397449518


In [390]:
test_pred = catmodel1000.predict(Test_Copy[feature_columns])

In [391]:
submission = pd.DataFrame({"uniqueid": Test_Copy["uniqueid"],
                                "bank_account": test_pred
                             })

In [392]:
submission.head()

,uniqueid,bank_account
0,uniqueid_6056,1.0
1,uniqueid_6060,1.0
2,uniqueid_6065,0.0
3,uniqueid_6072,0.0
4,uniqueid_6073,0.0


In [393]:
submission["uniqueid"] = Test["uniqueid"] + " x " + Test["country"]

In [394]:
submission.head()

,uniqueid,bank_account
0,uniqueid_6056 x Kenya,1.0
1,uniqueid_6060 x Kenya,1.0
2,uniqueid_6065 x Kenya,0.0
3,uniqueid_6072 x Kenya,0.0
4,uniqueid_6073 x Kenya,0.0


In [395]:
submission.to_csv("Zindi.csv", index=False)

/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 